In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Fundamentos de desoxidación de aceros

**Dr. Edgar Ivan Castro Cedeño**

[edgar.castro@cinvestav.mx](mailto:edgar.castro@cinvestav.mx)

## Contexto metalúrgico

Al terminar el proceso de refinación primaria del acero (BOF/EAF), se tiene un acero que contiene grandes cantidades de oxígeno disuelto en el baño. El orden de magnitud es:

$$a_O > 300 \, ppm$$

Una de las étapas en la refinación secundaria del acero es la desoxidación del baño, que se puede realizar mediante la adición de un agente desoxidante o mediante un tratamiento de vacío.

Este documento se enfoca en presentar conceptos básicos de desoxidación mediante la adición de agentes desoxidantes, y la aplicación de los mismos en el proceso de refinación secundaria de aceros.

## Reacción general de desoxidación

De forma general, la reacción de desoxidación de un acero se puede expresar como:

$$\left(M_xO_y\right) = x\left[M\right] + y\left[O_x\right] $$

$$K = \frac{a_M^x a_O^y}{a_{M_xO_y}} $$

where $a_{M_xO_y}$, $a_M^x$, y $a_O^y$, son las actividades termodinámicas de las especies químicas de interés.

### Estados de referencia

Para los **productos de desoxidación** se utiliza la substancia pura como estado de referencia:

$$a_{M_xO_y} = \gamma_{M_xO_y} {X_{M_xO_y}} $$
    
| Símbolo | Descripción |
| --- | --- |
|$a_{M_xO_y}$ | Actividad del producto de desoxidación $M_xO_y$  |   
|$\gamma_{M_xO_y}$ | Coeficiente de actividad Raoultiano  |
|$X_{M_xO_y}$ | Contenido de $M_xO_y$ en el producto de desoxidación, en fracción molar. |

En este documento no se habla de métodos para estimar la actividad de los productos de desoxidación en sistemas complejos.

Para los **solutos en el baño de metal fundido** se utiliza la solución Henriana con dilución al 1% en masa como estado de referencia:

$$h_M = f_M {\left[\%M\right]} $$

| Símbolo | Descripción |
| --- | --- |
|$h_M$ | actividad del soluto $M$ |
|$f_M$ | Coeficiente de actividad henriana |
|$\left[ \%M \right]$ | Contenido del soluto $M$ en el baño, in porcentaje másico.   |

### Formalismo de Wagner de coeficientes de interacción

Para considerar la dependencia entre la composición del baño y los coeficientes de actividad de los solutos, se va a utilizar el formalismo de Wagner.

$$\log f_i = \sum\limits_{j=2}^{n}{e_i^j \left[\%j\right]} 
        + \sum\limits_{j=2}^n{r_i^j \left[ \%j\right]^2} 
        + \sum\limits_{j=2}^{n-1} \sum\limits_{k>j}^{n} {r_i^{j,k} \left[\%j\right]\left[\%k\right]}$$
        
        
| Símbolo | Descripción |
| --- | --- |
|$e_i^j$     | Coeficiente de interacción de primer orden para el soluto $i$ en presencia del soluto $j$ |
|$r_i^j$     | Coeficiente de interacción de segundo orden para el soluto $i$ en presencia del soluto $j$ |
|$r_i^{j,k}$ | Coeficiente de interacción de segundo orden para el soluto $i$ en presencia de los soluto $j$, $k$ |

Aunque este formalismo fue definido originalmente para soluciones en dilución infinita, ha probado su utilidad práctica para el tratamiento termodinámico de soluciones con una concentración baja de solutos, como el hierro y el acero. Los parámetros de interacción de orden superior se utilizan para tratar casos como los aceros aleados, en los que la concentración de solutos es más alta.

## Desoxidación con aluminio

### Reacción de desoxidación

La reacción de deoxidación con alumino está dada por:

$$\left( Al_{2}O_{3}\right) = 2 \left[Al\right] + 3 \left[O\right] $$

$$K_{Al} =  \frac{h_{Al}^2  h_{O}^3}{a_{Al_2O_3}} $$



![Alumina inclusion](images/alumina1.png "Inclusión de alumina")

Aplicando leyes de logaritmos, la constante se puede escribir como:

$$\log K = 2 \log f_{Al} + 2 \log \left[\%Al\right] + 3 \log f_{O} + 3 \log \left[\%O \right] - \log a_{Al_2O_3} $$


Donde:

$\log f_{Al} = e_{Al}^{Al} \left[\%Al\right] + e_{Al}^{O} \left[\%O\right] + r_{Al}^{Al,Al} \left[\%Al\right]^2 + r_{Al}^{O,O} \left[\%O\right]^2 + r_{Al}^{Al,O} \left[\%Al\right]\left[\%O\right] $

$\log f_{O} = e_{O}^{Al} \left[\%Al\right] + e_{O}^{O} \left[\%O\right] + r_{O}^{Al,Al} \left[\%Al\right]^2 + r_{O}^{O,O} \left[\%O\right]^2 + r_{O}^{Al,O} \left[\%Al\right]\left[\%O\right] $


### Valor numérico de la constante de equilibrio

Donde:

$\log K_{Al} = -\frac{45300}{T} + 11.62 $

In [4]:
def logKAl(T: float) -> float:
    """
    logaritmo de la Constante de equilibrio 
    """
    return -45300 / T + 11.62

### Valor numérico de los coeficientes de interacción

#### Coeficientes de primer orden


$$\begin{bmatrix}e_{Al}^{Al} & e_{Al}^{O} \\ e_{O}^{Al} & e_{O}^{O}\end{bmatrix} = \begin{bmatrix}\frac{80.5}{T} & 3.21 - \frac{9720}{T} \\ 1.90 - \frac{5750}{T} & 0.76 - \frac{1750}{T}\end{bmatrix}$$

In [14]:
def e_Al_Al(T: float) -> float:
    return 80.5 / T

def e_Al_O(T: float) -> float:
    return 3.21 - 9720 / T

def e_O_Al(T: float) -> float:
    return 1.90 - 5750 / T

def e_O_O(T: float) -> float:
    return 0.76 - 1750/T

#### Coeficientes de segundo orden


$$\begin{bmatrix}r_{Al}^{Al, Al} & r_{Al}^{O,O} \\ r_{O}^{Al, Al} & r_{O}^{O,O}\end{bmatrix} = \begin{bmatrix}0 &  -107 - \frac{2.75\times10^{5}}{T} \\ 0.0033 - \frac{25.0}{T} & 0\end{bmatrix}$$

$$\begin{bmatrix} r_{Al}^{Al, O} \\ r_{O}^{Al, O}\end{bmatrix} = \begin{bmatrix} -0.021 - \frac{13.78}{T} \\ 127.3 + \frac{3.273\times10^{5}}{T} \end{bmatrix}$$

In [13]:
def r_Al_Al(T: float) -> float:
    return 0

def r_Al_O(T: float) -> float:
    return -107 - 2.75e5 / T

def r_O_Al(T: float) -> float:
    return 0.0033 - 25 / T

def r_O_O(T: float) -> float:
    return 0

def r_Al_AlO(T: float) -> float:
    return -0.021 -13.78 / T

def r_O_AlO(T: float) -> float:
    return 127.3 + 3.273e5 / T

## Construcción de curvas de equilibrio para la desoxidación con aluminio